In [39]:
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import col, when, sum as spark_sum
import zipfile
import os

# Buscar una relación entre el Covid y las "Cormobilidades"

### Descomprimir el zip para no guardarlo en el git

Ese lo tengo guardado en una carpeta en el escritorio:
- Datos/Covid_limpio.csv


In [ ]:
def descomprimir_zip(ruta_zip, carpeta_destino):
    # Crear carpeta si no existe
    os.makedirs(carpeta_destino, exist_ok=True)
    
    try:
        with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip:
            # Extraer todo
            archivo_zip.extractall(carpeta_destino)
            
            # Mostrar información
            print(f"Archivo descomprimido: {ruta_zip}")
            return True
            
    except Exception as e:
        print(f"ERROR: {e}")
        return False


descomprimir_zip("archivo.zip", "Data/")

ERROR: [Errno 2] No such file or directory: 'archivo.zip'


False

### Creción de un schema

In [22]:
mi_esquema = StructType([
    StructField('FECHA_ACTUALIZACION', DateType(), True),
    StructField('ID_REGISTRO', StringType(), True),
    StructField('ORIGEN', StringType(), True),
    StructField('SECTOR', StringType(), True),
    StructField('ENTIDAD_UM', IntegerType(), True),
    StructField('SEXO', IntegerType(), True),
    StructField('ENTIDAD_NAC', IntegerType(), True),
    StructField('ENTIDAD_RES', IntegerType(), True),
    StructField('MUNICIPIO_RES', IntegerType(), True),
    StructField('FECHA_INGRESO', DateType(), True),
    StructField('FECHA_SINTOMAS', DateType(), True),
    StructField('FECHA_DEF', DateType(), True),
    StructField('INTUBADO', IntegerType(), True),
    StructField('NEUMONIA', IntegerType(), True),
    StructField('EDAD', IntegerType(), True),
    StructField('NACIONALIDAD', IntegerType(), True),
    StructField('EMBARAZO', IntegerType(), True),
    StructField('HABLA_LENGUA_INDIG', IntegerType(), True),
    StructField('INDIGENA', IntegerType(), True),
    StructField('DIABETES', IntegerType(), True),
    StructField('ASMA', IntegerType(), True),
    StructField('INMUSUPR', IntegerType(), True),
    StructField('HIPERTENSION', IntegerType(), True),
    StructField('OTRA_COM', IntegerType(), True),
    StructField('CARDIOVASCULAR', IntegerType(), True),
    StructField('OBESIDAD', IntegerType(), True),
    StructField('RENAL_CRONICA', IntegerType(), True),
    StructField('OTRO_CASO', IntegerType(), True),
    StructField('TOMA_MUESTRA_LAB', IntegerType(), True),
    StructField('RESULTADO_LAB', IntegerType(), True),
    StructField('TOMA_MUESTRA_ANTIGENO', IntegerType(), True),
    StructField('RESULTADO_ANTIGENO', IntegerType(), True),
    StructField('CLASIFICACION_FINAL', IntegerType(), True),
    StructField('MIGRANTE', IntegerType(), True),
    StructField('PAIS_NACIONALIDAD', StringType(), True),
    StructField('PAIS_ORIGEN', StringType(), True),
    StructField('UCI', IntegerType(), True)])


In [23]:
spark = SparkSession.builder.appName('COVID19').config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()

ruta = 'Data/Covid_limpio.csv'
df = spark.read.csv(
    ruta,
    header = True,
    schema = mi_esquema,    # evita que Spark adivine tipos (así no se confunde)
    mode = "DROPMALFORMED", # elimina filas corruptas desde el inicio
    multiLine = False,      # evita que las comas o saltos de línea rompan filas
    escape="\"",          # ignora comillas mal cerradas
)
df.show()

25/12/06 18:19:16 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 40, schema size: 37
CSV file: file:///home/jair/Escritorio/Big%20Data/Proyectos/Covid/Data/Covid_limpio.csv


+-------------------+-----------+------+------+----------+----+-----------+-----------+-------------+-------------+--------------+---------+--------+--------+----+------------+--------+------------------+--------+--------+----+--------+------------+--------+--------------+--------+-------------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|FECHA_ACTUALIZACION|ID_REGISTRO|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|FECHA_DEF|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+-------------------+-----------+------+------+----------+----+-----------+-----------+-------------+-------------+-

In [24]:
spark = SparkSession.builder.appName('embarazos').getOrCreate()

df = spark.read.csv(
    ruta, 
    header = True,
    inferSchema=True,
    mode = 'PERMISSIVE',
    multiLine = False,
    escape = "\""
)

df.show()

25/12/06 18:21:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------------------+-----------+------+------+----------+----+-----------+-----------+-------------+-------------+-------------+--------------+----------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|FECHA_ACTUALIZACION|ID_REGISTRO|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|FECHA_INGRESO|FECHA_SINTOMAS| FECHA_DEF|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+-------------------+-----------+------+------+-------

## Crear la spark session

In [25]:
spark = SparkSession.builder.appName('Embarazos').getOrCreate()

ruta = 'Data/Covid_limpio.csv'
df = spark.read.csv(
    ruta, 
    header = True,
    inferSchema = True,
    mode = 'PERMISSIVE',
    multiLine = False,
    escape = "\""
)

df.show()

25/12/06 18:21:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------------------+-----------+------+------+----------+----+-----------+-----------+-------------+-------------+-------------+--------------+----------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|FECHA_ACTUALIZACION|ID_REGISTRO|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|FECHA_INGRESO|FECHA_SINTOMAS| FECHA_DEF|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+-------------------+-----------+------+------+-------

### ELiminar unas columnas de basura

In [26]:
fechas = [col for col in df.columns if 'FECHA_' in col]
eliminar = ['ID_REGISTRO'] + fechas
df = df.drop(*eliminar)
df.show()

+------+------+----------+----+-----------+-----------+-------------+-------------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+------+------+----------+----+-----------+-----------+-------------+-------------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+-

In [27]:
df.dtypes

[('ORIGEN', 'int'),
 ('SECTOR', 'int'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'int'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'int'),
 ('TIPO_PACIENTE', 'int'),
 ('INTUBADO', 'int'),
 ('NEUMONIA', 'int'),
 ('EDAD', 'int'),
 ('NACIONALIDAD', 'int'),
 ('EMBARAZO', 'int'),
 ('HABLA_LENGUA_INDIG', 'int'),
 ('INDIGENA', 'int'),
 ('DIABETES', 'int'),
 ('EPOC', 'int'),
 ('ASMA', 'int'),
 ('INMUSUPR', 'int'),
 ('HIPERTENSION', 'int'),
 ('OTRA_COM', 'int'),
 ('CARDIOVASCULAR', 'int'),
 ('OBESIDAD', 'int'),
 ('RENAL_CRONICA', 'int'),
 ('TABAQUISMO', 'int'),
 ('OTRO_CASO', 'int'),
 ('TOMA_MUESTRA_LAB', 'int'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'int'),
 ('RESULTADO_ANTIGENO', 'int'),
 ('CLASIFICACION_FINAL', 'int'),
 ('MIGRANTE', 'int'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'string'),
 ('UCI', 'int')]

### Registros por país

In [28]:
result = df.groupBy(df['PAIS_NACIONALIDAD']).count().orderBy('count', ascending = False).collect()
for row in result:
    print(f'{row[1]} - {row[0]}')

17186451 - MÃ©xico
51950 - Estados Unidos de AmÃ©rica
9242 - Venezuela
7410 - Colombia
5059 - RepÃºblica de Honduras
4412 - Guatemala
3193 - Cuba
2865 - Argentina
2835 - Nicaragua
2825 - EspaÃ±a
2750 - El Salvador
2703 - Brasil
2165 - Alemania
2104 - Otro
1903 - Francia
1847 - HaitÃ­
1795 - CanadÃ¡
1793 - PerÃº
1782 - China
1768 - Ecuador
1427 - JapÃ³n
1253 - Chile
1235 - Italia
946 - Ucrania
780 - Bolivia
623 - India
488 - RepÃºblica de Corea
451 - Gran BretaÃ±a (Reino Unido)
423 - RepÃºblica Dominicana
388 - Rusia
278 - Suiza
251 - RepÃºblica de Costa Rica
247 - Filipinas
235 - Belice
232 - Holanda
227 - RepÃºblica Oriental del Uruguay
209 - Australia
207 - Polonia
183 - RepÃºblica de PanamÃ¡
179 - Portugal
173 - Paraguay
146 - SE DESCONOCE
143 - Rumania
141 - Estado Libre Asociado de Puerto Rico
134 - BÃ©lgica
119 - Israel
112 - SudÃ¡frica
103 - Irlanda
98 - TurquÃ­a
95 - Austria
92 - Dinamarca
86 - Nigeria
84 - RepÃºblica Checa y RepÃºblica Eslovaca
75 - Zona Neutral
73 - Egipto
72

In [29]:
antes_null = df.count()

### Eliminarle los nulos al dataset

In [30]:
df = df.na.drop()
despues_null = df.count()
print(f'Datos nulos eliminados: {antes_null - despues_null}')

Datos nulos eliminados: 0


### ELIMINAR LO QUE NO QUIERO USAR

In [31]:
fechas = [col for col in df.columns if 'FECHA_' in col]
eliminar = ['ID_REGISTRO', 'INTUBADO'] + fechas
df = df.drop(*eliminar)
df.show()

+------+------+----------+----+-----------+-----------+-------------+-------------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+------+------+----------+----+-----------+-----------+-------------+-------------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+----

### Cambiarle los datos a numéricos para trabajarlos

In [32]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

numericas = [col for col in df.columns if col != 'PAIS_NACIONALIDAD']
for c in numericas:
    df = df.withColumn(c, col(c).cast(IntegerType()))

df.dtypes

[('ORIGEN', 'int'),
 ('SECTOR', 'int'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'int'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'int'),
 ('TIPO_PACIENTE', 'int'),
 ('NEUMONIA', 'int'),
 ('EDAD', 'int'),
 ('NACIONALIDAD', 'int'),
 ('EMBARAZO', 'int'),
 ('HABLA_LENGUA_INDIG', 'int'),
 ('INDIGENA', 'int'),
 ('DIABETES', 'int'),
 ('EPOC', 'int'),
 ('ASMA', 'int'),
 ('INMUSUPR', 'int'),
 ('HIPERTENSION', 'int'),
 ('OTRA_COM', 'int'),
 ('CARDIOVASCULAR', 'int'),
 ('OBESIDAD', 'int'),
 ('RENAL_CRONICA', 'int'),
 ('TABAQUISMO', 'int'),
 ('OTRO_CASO', 'int'),
 ('TOMA_MUESTRA_LAB', 'int'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'int'),
 ('RESULTADO_ANTIGENO', 'int'),
 ('CLASIFICACION_FINAL', 'int'),
 ('MIGRANTE', 'int'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'int'),
 ('UCI', 'int')]

### Preparar columnas para un análisis estadístico

In [ ]:
def crear_variables_cormobilidad(df):
    condiciones = ['DIABETES', 'HIPERTENSION', 'OBESIDAD', 'ASMA', 'EPOC', 'INMUSUPR', 'RENAL_CRONICA', 'TABAQUISMO']

    # Ponerles un contador
    df = df.withColumn("N_CORMOBILIDADES", 
                       when(col('DIABETES') == 1, 1).otherwise(0) +
                       when(col('HIPERTENSION') == 1, 1).otherwise(0) + 
                       when(col('OBESIDAD') == 1, 1).otherwise(0) + 
                       when(col('ASMA') == 1, 1).otherwise(0) + 
                       when(col('EPOC') == 1, 1).otherwise(0) + 
                       when(col('INMUSUPR') == 1, 1).otherwise(0) + 
                       when(col('RENAL_CRONICA') == 1, 1).otherwise(0) + 
                       when(col('TABAQUISMO') == 1, 1).otherwise(0))
    
    # Ponerles una categoría
    df = df.withColumn("Corm_cardiovascular",
                       when(col("HIPERTENSION") == 1 | col("DIABETES") == 1 | col("OBESIDAD") == 1, 1).otherwise(0)) 
    
    df = df.withColumn("Corm_respiratoria",
                       when(col("ASMA") == 1 | col('EPOC') == 1), 1).otherwise(0)
    
    df = df.whithColumn('Corm_inmunosup',
                        when(col("INMUSUPR") == 1), 1).otherwise(0)
    
    df = df.whithColumn

In [ ]:
from pyspark.sql.functions import col, when, sum as spark_sum

def crear_variables_comorbilidad(df):
    """
    Crea variables categóricas para análisis de comorbilidades.
    """
    
    # 1. Contador de comorbilidades por paciente
    condiciones = ['DIABETES', 'HIPERTENSION', 'OBESIDAD', 'ASMA', 
                   'EPOC', 'INMUSUPR', 'RENAL_CRONICA', 'TABAQUISMO']
    
    # Inicializar contador
    df = df.withColumn("NUM_COMORBILIDADES", 
                       when(col("DIABETES") == 1, 1).otherwise(0) +
                       when(col("HIPERTENSION") == 1, 1).otherwise(0) +
                       when(col("OBESIDAD") == 1, 1).otherwise(0) +
                       when(col("ASMA") == 1, 1).otherwise(0) +
                       when(col("EPOC") == 1, 1).otherwise(0) +
                       when(col("INMUSUPR") == 1, 1).otherwise(0) +
                       when(col("RENAL_CRONICA") == 1, 1).otherwise(0) +
                       when(col("TABAQUISMO") == 1, 1).otherwise(0))
    
    # 2. Crear categorías
    df = df.withColumn("TIENE_CARDIOVASCULAR",
                       when((col("HIPERTENSION") == 1) | 
                            (col("DIABETES") == 1) | 
                            (col("OBESIDAD") == 1), 1)
                       .otherwise(0))
    
    df = df.withColumn("TIENE_RESPIRATORIA",
                       when((col("ASMA") == 1) | 
                            (col("EPOC") == 1), 1)
                       .otherwise(0))
    
    df = df.withColumn("TIENE_INMUNOSUPRESION",
                       when(col("INMUSUPR") == 1, 1)
                       .otherwise(0))
    
    df = df.withColumn("TIENE_RENAL_CRONICA",
                       when(col("RENAL_CRONICA") == 1, 1)
                       .otherwise(0))
    
    # 3. Categoría de riesgo según comorbilidades
    df = df.withColumn("CATEGORIA_RIESGO",
                       when(col("NUM_COMORBILIDADES") >= 3, "ALTO")
                       .when(col("NUM_COMORBILIDADES") == 2, "MEDIO")
                       .when(col("NUM_COMORBILIDADES") == 1, "BAJO")
                       .otherwise("NINGUNO"))
    
    return df

# Uso:
df = crear_variables_comorbilidad(df)

## Experimentar con los groupBy

Quiero saber:

* Cuántas mujeres hay por cada país
* Cuántas mujeres hay en total en comparación con los hombres
* Cuántas mujeres se infectaron y cuantas no
* Cuántas mujeres estaban embarazadas y cuántas no
* Cuantas de mujeres están embarazadas estaban contagiadas

In [33]:
'''
Cuántas mujeres hay de cada país
'''
n_mujeres = df.filter(df['SEXO'] == 1).count()
print(f'Hay un total de {n_mujeres} mujeres')
df.filter(df['SEXO'] == 1).groupBy('PAIS_NACIONALIDAD').count().orderBy('count', ascending = False).show()

Hay un total de 9291799 mujeres


+--------------------+-------+
|   PAIS_NACIONALIDAD|  count|
+--------------------+-------+
|             MÃ©xico|9234465|
|Estados Unidos de...|  25705|
|           Venezuela|   5036|
|            Colombia|   4019|
|RepÃºblica de Hon...|   1989|
|           Guatemala|   1804|
|                Cuba|   1433|
|              Brasil|   1252|
|         El Salvador|   1248|
|           Argentina|   1243|
|           Nicaragua|   1036|
|             EspaÃ±a|   1034|
|                Otro|    875|
|               PerÃº|    866|
|             Ecuador|    819|
|              HaitÃ­|    807|
|             CanadÃ¡|    798|
|             Francia|    790|
|            Alemania|    691|
|               Chile|    558|
+--------------------+-------+
only showing top 20 rows


In [34]:
'''
Cuántas mujeres hay en total en comparación con los hombres
'''
total = df.count()
n_mujeres = df.filter(df['SEXO'] == 1).count()
n_hombres = df.filter(df['SEXO'] == 2).count()

prop_m = n_mujeres / total
prop_h = n_hombres / total

print(f'La proporción de mujeres: {np.round(prop_m * 100, 2)} %')
print(f'La proporción de hombres: {np.round(prop_h * 100, 2)} %')

La proporción de mujeres: 53.68 %
La proporción de hombres: 46.32 %


In [35]:
from pyspark.sql.functions import col, expr

# 1. Convertir a bigint usando try_cast
df = df.withColumn("CLASIFICACION_FINAL_INT", expr("try_cast(CLASIFICACION_FINAL as bigint)"))

# 2. Filtrar solo valores válidos y materializar
df_validos = df.filter(col("CLASIFICACION_FINAL_INT").isNotNull()).cache()

# 3. Contar mujeres con clasificación positiva usando la columna casteada
positivos_covid = [1, 2, 3]
mujeres_positivas = df_validos.filter(
    (col('SEXO') == 1) & (col('CLASIFICACION_FINAL_INT').isin(positivos_covid))).count()

print(mujeres_positivas)


25/12/06 18:24:26 WARN CacheManager: Asked to cache already cached data.
25/12/06 18:24:26 WARN BlockManager: Putting block rdd_89_10 failed due to exception org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'El Salvador' of the type "STRING" cannot be cast to "INT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"cast" was called from
line 6 in cell [9]
.
25/12/06 18:24:26 WARN BlockManager: Block rdd_89_10 could not be removed as it was not found on disk or in memory
25/12/06 18:24:26 WARN BlockManager: Putting block rdd_89_7 failed due to exception org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'RepÃºblica de Honduras' of the type "STRING" cannot be cast to "INT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed inp

NumberFormatException: [CAST_INVALID_INPUT] The value 'RepÃºblica de Honduras' of the type "STRING" cannot be cast to "INT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"cast" was called from
line 6 in cell [9]
